<h1>Aggregations<h1>

In [1]:
import pandas as pd
import sqlite3

<h2>create a connection to the database using the library sqlite3<h2>

In [3]:
conn = sqlite3.connect("../../data/checking-logs.sqlite")

<h2>get the schema of the table test<h2>

In [5]:
pd.io.sql.read_sql("PRAGMA table_info(test);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


<h2>get only the first 10 rows of the table test to check what the table looks like<h2>

In [6]:
pd.io.sql.read_sql("SELECT * FROM test LIMIT 10", conn)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


<h2>find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query<h2>

In [33]:
query = '''
    SELECT uid, (unixepoch(first_commit_ts) - deadlines)/3600 AS delta
    FROM test
        JOIN deadlines ON  test.labname = deadlines.labs
    WHERE labs <> 'project1'
    ORDER BY delta ASC
    LIMIT 1

'''
df_min = pd.io.sql.read_sql(query, conn)

<h2>do the same thing, but for the maximum, using only one query, the dataframe name is df_max<h2>

In [35]:
query = '''
    SELECT uid, (unixepoch(first_commit_ts) - deadlines)/3600 AS delta
    FROM test
        JOIN deadlines ON  test.labname = deadlines.labs
    WHERE labs <> 'project1'
    ORDER BY delta DESC
    LIMIT 1

'''
df_max = pd.io.sql.read_sql(query, conn)

<h2>do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg<h2>

In [ ]:
query = '''
    SELECT AVG(unixepoch(first_commit_ts) - deadlines)/3600 AS delta
    FROM test
        JOIN deadlines ON  test.labname = deadlines.labs
    WHERE labs <> 'project1'

'''
df_avg = pd.io.sql.read_sql(query, conn)

<h2>we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference<h2>

In [62]:
query = '''
    SELECT test.uid, AVG(unixepoch(first_commit_ts)-deadlines) AS avg_diff, 
        views AS pageviews
    FROM test
        JOIN deadlines ON test.labname = deadlines.labs
        JOIN (
            SELECT pageviews.uid, COUNT(datetime) AS views
            FROM pageviews
            GROUP BY uid
        ) AS pageviews_per_user ON test.uid = pageviews_per_user.uid
    WHERE labs <> 'project1'
    GROUP BY test.uid

'''
views_diff = pd.io.sql.read_sql(query, conn)

In [64]:
views_diff[["avg_diff", "pageviews"]].corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279143
pageviews,-0.279143,1.000000


<h2>close the connection<h2>

In [65]:
conn.close()